In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC50_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 1
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC50(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC50(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC50(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料
讀取3秒資料
讀取5秒資料
Data Loading Time: 0.25 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []


#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ", fold)
    print("----------------------------------------------------------------------------------------------------------------")

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=50)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    #------------------------------------------------
    #需要調整的地方
    #------------------------------------------------
    #學習率
    lr= 0.00032
    #epoch
    epoch= 120
    #訓練資料(要用幾秒訓練) data_1S/data_3S/data_5S
    trainData = data_3S
    #------------------------------------------------
    
    #紀錄訓練開始時間
    start_time = time.time()

    #訓練模型
    model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold,
                                                                setLR=lr,
                                                                setEpoch=epoch)
    
    #測試準確度
    test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
    test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
    test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

    #紀錄訓練結束時間
    end_time = time.time()

    #計算訓練耗時
    training_time = end_time - start_time

    #印出訓練耗時
    print("Training Time: {:.2f} seconds".format(training_time))

    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("訓練過程\n")
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    for i in range(len(train_loss)):
        # 寫入資料
        f.write("Fold: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
    f.write("Training Time: {:.2f} seconds\n".format(training_time))
    f.write("---------------------------------------------------------------------------------------------------------------\n")

    #--------------------------------------------------------------------

    
    kFoldTrainTime.append(training_time)
    kFoldTrainAccuracy.append(train_accuracy[-1])
    kFoldTestAccuracy1.append(test_accuracy1)
    kFoldTestAccuracy3.append(test_accuracy3)
    kFoldTestAccuracy5.append(test_accuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

----------------------------------------------------------------------------------------------------------------
fold:  1
----------------------------------------------------------------------------------------------------------------
可用的 GPU 数量: 1
Train Epoch: 1/120 Train_Loss: 1.749406337738037 Train_acc: 28.583334%
Train Epoch: 2/120 Train_Loss: 0.9872839450836182 Train_acc: 59.562500%
Train Epoch: 3/120 Train_Loss: 0.6827992796897888 Train_acc: 74.583336%
Train Epoch: 4/120 Train_Loss: 0.49667301774024963 Train_acc: 84.583336%
Train Epoch: 5/120 Train_Loss: 0.3354400098323822 Train_acc: 90.083336%
Train Epoch: 6/120 Train_Loss: 0.5546604990959167 Train_acc: 92.354172%
Train Epoch: 7/120 Train_Loss: 0.438840389251709 Train_acc: 92.333336%
Train Epoch: 8/120 Train_Loss: 0.21372610330581665 Train_acc: 93.458336%
Train Epoch: 9/120 Train_Loss: 0.4173711836338043 Train_acc: 94.166672%
Train Epoch: 10/120 Train_Loss: 0.19133791327476501 Train_acc: 93.604172%
Train Epoch: 11/120 Train_Los

Train Epoch: 109/120 Train_Loss: 0.058022938668727875 Train_acc: 97.437500%
Train Epoch: 110/120 Train_Loss: 0.07500994205474854 Train_acc: 97.458336%
Train Epoch: 111/120 Train_Loss: 0.012669901363551617 Train_acc: 97.458336%
Train Epoch: 112/120 Train_Loss: 0.03663463518023491 Train_acc: 97.437500%
Train Epoch: 113/120 Train_Loss: 0.006391832139343023 Train_acc: 97.437500%
Train Epoch: 114/120 Train_Loss: 0.147785022854805 Train_acc: 97.458336%
Train Epoch: 115/120 Train_Loss: 0.046523772180080414 Train_acc: 97.437500%
Train Epoch: 116/120 Train_Loss: 0.18710705637931824 Train_acc: 97.416672%
Train Epoch: 117/120 Train_Loss: 0.16259869933128357 Train_acc: 97.458336%
Train Epoch: 118/120 Train_Loss: 0.24117225408554077 Train_acc: 97.333336%
Train Epoch: 119/120 Train_Loss: 0.07968846708536148 Train_acc: 97.041672%
Train Epoch: 120/120 Train_Loss: 0.09220902621746063 Train_acc: 97.208336%
Test accuracy of the model: 42.050000%
Test accuracy of the model: 57.333333%
Test accuracy of the

Train Epoch: 95/120 Train_Loss: 0.1131034716963768 Train_acc: 97.395836%
Train Epoch: 96/120 Train_Loss: 0.03256399184465408 Train_acc: 97.416672%
Train Epoch: 97/120 Train_Loss: 0.08754664659500122 Train_acc: 97.395836%
Train Epoch: 98/120 Train_Loss: 0.08807535469532013 Train_acc: 97.312500%
Train Epoch: 99/120 Train_Loss: 0.14882446825504303 Train_acc: 97.416672%
Train Epoch: 100/120 Train_Loss: 0.13561005890369415 Train_acc: 97.416672%
Train Epoch: 101/120 Train_Loss: 0.06937264651060104 Train_acc: 97.416672%
Train Epoch: 102/120 Train_Loss: 0.054613739252090454 Train_acc: 97.395836%
Train Epoch: 103/120 Train_Loss: 0.06267672032117844 Train_acc: 97.395836%
Train Epoch: 104/120 Train_Loss: 0.006063664332032204 Train_acc: 97.416672%
Train Epoch: 105/120 Train_Loss: 0.09003941714763641 Train_acc: 97.312500%
Train Epoch: 106/120 Train_Loss: 0.0893777534365654 Train_acc: 97.416672%
Train Epoch: 107/120 Train_Loss: 0.24914930760860443 Train_acc: 97.416672%
Train Epoch: 108/120 Train_Los

Train Epoch: 81/120 Train_Loss: 0.04749758914113045 Train_acc: 97.562500%
Train Epoch: 82/120 Train_Loss: 0.004749210085719824 Train_acc: 97.541672%
Train Epoch: 83/120 Train_Loss: 0.040642380714416504 Train_acc: 97.395836%
Train Epoch: 84/120 Train_Loss: 0.09925708174705505 Train_acc: 97.458336%
Train Epoch: 85/120 Train_Loss: 0.09893029183149338 Train_acc: 97.541672%
Train Epoch: 86/120 Train_Loss: 0.0077075655572116375 Train_acc: 97.583336%
Train Epoch: 87/120 Train_Loss: 0.13748176395893097 Train_acc: 97.520836%
Train Epoch: 88/120 Train_Loss: 0.08627340197563171 Train_acc: 97.562500%
Train Epoch: 89/120 Train_Loss: 0.00834079459309578 Train_acc: 97.562500%
Train Epoch: 90/120 Train_Loss: 0.12282243371009827 Train_acc: 95.854172%
Train Epoch: 91/120 Train_Loss: 0.04759090393781662 Train_acc: 97.333336%
Train Epoch: 92/120 Train_Loss: 0.08539784699678421 Train_acc: 97.583336%
Train Epoch: 93/120 Train_Loss: 0.08756799250841141 Train_acc: 97.604172%
Train Epoch: 94/120 Train_Loss: 0.

Train Epoch: 67/120 Train_Loss: 0.05759330838918686 Train_acc: 97.125000%
Train Epoch: 68/120 Train_Loss: 0.08790536224842072 Train_acc: 97.062500%
Train Epoch: 69/120 Train_Loss: 0.16480527818202972 Train_acc: 97.166672%
Train Epoch: 70/120 Train_Loss: 0.08313295990228653 Train_acc: 97.166672%
Train Epoch: 71/120 Train_Loss: 0.09547882527112961 Train_acc: 97.187500%
Train Epoch: 72/120 Train_Loss: 0.03524177521467209 Train_acc: 97.187500%
Train Epoch: 73/120 Train_Loss: 0.20826022326946259 Train_acc: 97.083336%
Train Epoch: 74/120 Train_Loss: 0.09942252933979034 Train_acc: 97.104172%
Train Epoch: 75/120 Train_Loss: 0.04138194024562836 Train_acc: 97.208336%
Train Epoch: 76/120 Train_Loss: 0.09348767250776291 Train_acc: 97.166672%
Train Epoch: 77/120 Train_Loss: 0.1534828543663025 Train_acc: 97.062500%
Train Epoch: 78/120 Train_Loss: 0.10162235051393509 Train_acc: 97.000000%
Train Epoch: 79/120 Train_Loss: 0.13529625535011292 Train_acc: 97.166672%
Train Epoch: 80/120 Train_Loss: 0.18938

Train Epoch: 53/120 Train_Loss: 0.12368819862604141 Train_acc: 97.437500%
Train Epoch: 54/120 Train_Loss: 0.15266406536102295 Train_acc: 96.916672%
Train Epoch: 55/120 Train_Loss: 0.21237201988697052 Train_acc: 93.812500%
Train Epoch: 56/120 Train_Loss: 0.24314764142036438 Train_acc: 95.312500%
Train Epoch: 57/120 Train_Loss: 0.16133570671081543 Train_acc: 97.187500%
Train Epoch: 58/120 Train_Loss: 0.044557828456163406 Train_acc: 97.312500%
Train Epoch: 59/120 Train_Loss: 0.1618250608444214 Train_acc: 97.270836%
Train Epoch: 60/120 Train_Loss: 0.039061278104782104 Train_acc: 97.437500%
Train Epoch: 61/120 Train_Loss: 0.03770163282752037 Train_acc: 97.375000%
Train Epoch: 62/120 Train_Loss: 0.046564843505620956 Train_acc: 97.354172%
Train Epoch: 63/120 Train_Loss: 0.13660748302936554 Train_acc: 97.395836%
Train Epoch: 64/120 Train_Loss: 0.10981732606887817 Train_acc: 97.375000%
Train Epoch: 65/120 Train_Loss: 0.17921042442321777 Train_acc: 97.333336%
Train Epoch: 66/120 Train_Loss: 0.12

In [5]:
#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    # 寫入資料
    f.write("Fold: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, kFoldTrainAccuracy[fold], kFoldTestAccuracy1[fold], kFoldTestAccuracy3[fold], kFoldTestAccuracy5[fold], kFoldTrainTime[fold]))
    f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(max(kFoldTrainAccuracy), max(kFoldTestAccuracy1), max(kFoldTestAccuracy3), max(kFoldTestAccuracy5)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(min(kFoldTrainAccuracy), min(kFoldTestAccuracy1), min(kFoldTestAccuracy3), min(kFoldTestAccuracy5)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(np.mean(kFoldTrainAccuracy), np.mean(kFoldTestAccuracy1), np.mean(kFoldTestAccuracy3), np.mean(kFoldTestAccuracy5)))

f.close()